## W266 Final Project

In [3]:
import numpy as np
import pandas as pd
import re
import glob #this makes a list of all the files in a directory
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [72]:
files_list = glob.glob("./data/imsdb_scenes_dialogs_nov_2015/dialogs/Adventure/a*_dialog.txt")

file_array = np.array(files_list)
files = pd.DataFrame(file_array.reshape(-1,1), columns=["FilePath"])

title_list = []
genre_list = []
for i in files.index:
    genretitle = files['FilePath'][i][45:] #get rid of all the ./data/imsdb...
    a, b = genretitle.split("/")
    genre_list.append(a)
    title_list.append(b[:-11]) #take off '_dialog.txt' from the file name
    
    
files['Genre']=genre_list
files['Title']=title_list

files

,FilePath,Genre,Title
0,./data/imsdb_scenes_dialogs_nov_2015/dialogs/A...,Adventure,abyssthe
1,./data/imsdb_scenes_dialogs_nov_2015/dialogs/A...,Adventure,avengersthe
2,./data/imsdb_scenes_dialogs_nov_2015/dialogs/A...,Adventure,airforceone
3,./data/imsdb_scenes_dialogs_nov_2015/dialogs/A...,Adventure,armyofdarkness
4,./data/imsdb_scenes_dialogs_nov_2015/dialogs/A...,Adventure,antz
5,./data/imsdb_scenes_dialogs_nov_2015/dialogs/A...,Adventure,avatar
6,./data/imsdb_scenes_dialogs_nov_2015/dialogs/A...,Adventure,avengersthe2012


In [6]:
movie1 = pd.read_fwf('./data/imsdb_scenes_dialogs_nov_2015/dialogs/Adventure/avatar_dialog.txt',
                    header=None, names=["RawLine"],encoding='utf')
movie2 = pd.read_fwf('./data/imsdb_scenes_dialogs_nov_2015/dialogs/Adventure/1492conquestofparadise_dialog.txt',
                    header=None, names=["RawLine"],encoding='utf')
### suggestion: always read in movies as UTF-8 so we don't have weird “&amp;amp;”  instead of &

In [102]:
def is_name(string):
    #string1 = re.sub('(V.O.)|CUT TO:','',string)
    ### nikki edit
    string1 = re.sub(r'\W+', '', re.sub('(V.O.)|CUT TO:','',string))
    n = re.match('[A-Z][A-Z]+', string1)
    if n: return True
    else: return False

def is_speech(string):
    if not is_name(string):
        return True
    else: return False

def is_cutto(string):
    n = re.match('CUT TO', string)
    #print n
    if n: return True
    else: return False
    

def parse_movie(movie, genre, title):   
#   Args:
#     movie is a pandas dataframe of the raw movie script input
#     genre is the movie genre from the folder it's read in from
#     title is the title (file name prior to _dialog.txt)
#   Output:
#     a dataframe of the form below
    #print genre, title
    curr_char = None
    line = 0
    title_column=[]
    genre_column=[]
    char_column=[]
    speech_column=[]
    line_column=[] 
    for i in movie.index:
        title_column.append(title) 
        genre_column.append(genre) 
        current = str(movie['RawLine'][i])
        #current = str(current)
        print current
        if current.lower() == 'nan':
            print 'stop'
                        
        elif is_cutto(current):
            char_column.append("")
            line_column.append(line)
            speech_column.append("")
        
        elif is_name(current):
            
            #starting a new character. hold onto this name for the next lines
            #until you see another new character name
            #also try to remove things that are irrelevant to the name 
            ### nikki edit: add in str.strip() in case voice over left whitespaces
            curr_char = re.sub('\(V.O.\)|V.O.|\(O.S.\)|O.S.|\(CONT\'D\)|CONT\'D','',current).strip()
            char_column.append(curr_char)
            
            line += 1
            line_column.append(line)
            speech_column.append("")
            
        elif is_speech(current):
            
            char_column.append(curr_char)
            line_column.append(line)
            speech_column.append(current)
            
    movie['Title']=title_column
    movie['Genre']=genre_column
    movie['Char']=char_column
    movie['LineID']=line_column
    movie['Speech']=speech_column

    return movie



parse_movie(movie1, 'Adventure', 'avatar')   
parse_movie(movie2, 'Adventure', '1492conquestofparadise')
movie1.head(n=15)
    
#Intermediate dataframe should look something like this:

#RawLine                                  Title     Genre      Char   LineID     Speech

#JAKE (V.O.)                              Avatar    Adventure  JAKE   1 
#They can fix a spinal, if you've got the Avatar    Adventure  JAKE   1          They can fix... 
#money. But not on vet benefits, not in   Avatar    Adventure  JAKE   1          money. But not...


#And then hopefully we can eventually somehow aggregate by LineID to:

#Title   Genre      Char  Speech
#Avatar  Adventure  JAKE  They can fix a spinal, if you've got the money. But not on vet...


AVATAR
Written by
James Cameron
VOICE (V.O.)
When I was lying there in the VA
hospital, with a big hole blown through
the middle of my life, I started having
these dreams of flying.
VOICE (V.O.)
Sooner or later though, you always have
to wake up...
CUT TO:
JAKE (V.O.)
They can fix a spinal, if you've got the
money. But not on vet benefits, not in
this economy.
PERKY NEWSCASTER
The Bengal tiger, extinct for over a
century, is making a comeback. These
cloned tiger cubs at the Beijing Zoo
are...
2.
JAKE (V.O.)
I became a Marine for the hardship. To
be hammered on the anvil of life. I told
myself I could pass any test a man can
pass.
CUT TO:
JAKE (V.O.)
Let's get it straight up front. I don't
want your pity. I know the world's a
cold-ass bitch.
JAKE (V.O.)
You want a fair deal, you're on the wrong
planet. The strong prey on the weak.
3.
JAKE (V.O.)
It's just the way things are.   And nobody
does a damn thing.
JAKE (V.O.)
All I ever wanted in my sorry-ass life
was a single thing worth fight

,RawLine,Title,Genre,Char,LineID,Speech
0,AVATAR,avatar,Adventure,AVATAR,1,
1,Written by,avatar,Adventure,AVATAR,1,Written by
2,James Cameron,avatar,Adventure,AVATAR,1,James Cameron
3,VOICE (V.O.),avatar,Adventure,VOICE,2,
4,When I was lying there in the VA,avatar,Adventure,VOICE,2,When I was lying there in the VA
5,"hospital, with a big hole blown through",avatar,Adventure,VOICE,2,"hospital, with a big hole blown through"
6,"the middle of my life, I started having",avatar,Adventure,VOICE,2,"the middle of my life, I started having"
7,these dreams of flying.,avatar,Adventure,VOICE,2,these dreams of flying.
8,VOICE (V.O.),avatar,Adventure,VOICE,3,
9,"Sooner or later though, you always have",avatar,Adventure,VOICE,3,"Sooner or later though, you always have"


In [41]:
movie1['Char'].value_counts()

JAKE                                       757
GRACE                                      434
QUARITCH                                   253
NEYTIRI                                    186
SELFRIDGE                                  137
NORM                                       123
(SUBTITLED)                                119
TRUDY                                       93
                                            87
MAX                                         69
MO'AT                                       63
TSU'TEY                                     55
WAINFLEET                                   33
EYTUKAN                                     18
PILOT                                       17
MED TECH                                    13
CREW CHIEF                                  13
TROOPER                                     11
JAKE (RECORDED)                             10
(QUIETLY)                                    8
VOICE                                        8
(ANGUISHED)  

In [9]:
movie1[movie1["Char"] == 'GRACE']

,RawLine,Title,Genre,Char,LineID,Speech
238,GRACE,avatar,Adventure,GRACE,61,
245,GRACE,avatar,Adventure,GRACE,64,
246,Norm. I hear good things about you.,avatar,Adventure,GRACE,64,Norm. I hear good things about you.
247,How's your Na'vi?,avatar,Adventure,GRACE,64,How's your Na'vi?
248,14.,avatar,Adventure,GRACE,64,14.
253,GRACE,avatar,Adventure,GRACE,66,
261,GRACE,avatar,Adventure,GRACE,71,
262,(turning to Jake),avatar,Adventure,GRACE,71,(turning to Jake)
263,"Yeah, yeah, I know who you are, and I",avatar,Adventure,GRACE,71,"Yeah, yeah, I know who you are, and I"
264,don't need you. I need your brother.,avatar,Adventure,GRACE,71,don't need you. I need your brother.


In [10]:
movie2.head(15)

,RawLine,Title,Genre,Char,LineID,Speech
0,<b><!--,1492conquestofparadise,Adventure,None,0,<b><!--
1,</b>if (window!= top),1492conquestofparadise,Adventure,None,0,</b>if (window!= top)
2,top.location.href=location.href,1492conquestofparadise,Adventure,None,0,top.location.href=location.href
3,<b>// -->,1492conquestofparadise,Adventure,None,0,<b>// -->
4,</b>,1492conquestofparadise,Adventure,None,0,</b>
5,1492: CONQUEST OF PARADISE,1492conquestofparadise,Adventure,None,0,1492: CONQUEST OF PARADISE
6,1492: CONQUEST OF PARADISE,1492conquestofparadise,Adventure,None,0,1492: CONQUEST OF PARADISE
7,by,1492conquestofparadise,Adventure,None,0,by
8,Roselyne Bosch,1492conquestofparadise,Adventure,None,0,Roselyne Bosch
9,REVISED,1492conquestofparadise,Adventure,REVISED,1,


In [11]:
movie2['Char'].value_counts()

COLUMBUS                                825
SANCHEZ                                 212
MARCHENA                                122
PINZON                                  100
AROJAZ                                  100
FERNANDO                                 92
ISABEL                                   92
SANTANGEL                                55
                                         51
MOXICA                                   51
AL                                       44
UTAPAN                                   40
BEATRIX                                  38
MENDEZ                                   32
BOBADILLA                                30
BARTOLOME                                29
BROTHER BUYL                             28
GIACOMO                                  24
SAILOR                                   22
ADMINISTRATOR                            21
DIEGO                                    20
SOLDIER                                  14
DIEGO DE DEZA                   

In [12]:
### keep only characters with at least 20 lines per movie

def top_chars(df, threshold = 20):
    #dframe = df
#     print dframe.head()
#     print type(dframe)
    threshold = threshold
    char_list = pd.DataFrame(df["Char"].value_counts())
    to_remove = char_list[char_list['Char'] <= threshold].index
    cl1 = char_list.drop(to_remove)
    cl1['line_count'] = cl1['Char']
    cl1['Char'] = cl1.index
    #char_list.reset_index(drop=True)
    query = 'SELECT d.*, c.line_count FROM df d INNER JOIN cl1 c on d.Char = c.Char'
    enriched_df = sqldf(query, locals())
    enriched_df = enriched_df[enriched_df['Char'] != '']
    #return cl1.head(20)
    return enriched_df

def pipeline(df,threshold=20):
    ### before top chars, we can add something about a proper title finder
    tc = top_chars(df, threshold)
    
    ### add something about the gender id
    ### add something about vocab enrichment
    ### output should be called final_df
    final_df = tc
    return tc
    

enriched_movie2 = pipeline(movie2)

In [13]:
enriched_movie2.head()

,RawLine,Title,Genre,Char,LineID,Speech,line_count
1,SANCHEZ,1492conquestofparadise,Adventure,SANCHEZ,2,,212
2,Your Majesty would wish to know the,1492conquestofparadise,Adventure,SANCHEZ,2,Your Majesty would wish to know the,212
3,true facts concerning the island of,1492conquestofparadise,Adventure,SANCHEZ,2,true facts concerning the island of,212
4,"Hispanola, our first settlement in",1492conquestofparadise,Adventure,SANCHEZ,2,"Hispanola, our first settlement in",212
5,"the New World, and the activities",1492conquestofparadise,Adventure,SANCHEZ,2,"the New World, and the activities",212


In [14]:
enriched_movie2['Char'].value_counts()

COLUMBUS         825
SANCHEZ          212
MARCHENA         122
AROJAZ           100
PINZON           100
FERNANDO          92
ISABEL            92
SANTANGEL         55
MOXICA            51
AL                44
UTAPAN            40
BEATRIX           38
MENDEZ            32
BOBADILLA         30
BARTOLOME         29
BROTHER BUYL      28
GIACOMO           24
SAILOR            22
ADMINISTRATOR     21
Name: Char, dtype: int64

In [15]:
enriched_movie1 = pipeline(movie1)

In [16]:
enriched_movie1.head()

,RawLine,Title,Genre,Char,LineID,Speech,line_count
1,JAKE (V.O.),avatar,Adventure,JAKE,4,,757
2,"They can fix a spinal, if you've got the",avatar,Adventure,JAKE,4,"They can fix a spinal, if you've got the",757
3,"money. But not on vet benefits, not in",avatar,Adventure,JAKE,4,"money. But not on vet benefits, not in",757
4,this economy.,avatar,Adventure,JAKE,4,this economy.,757
5,JAKE (V.O.),avatar,Adventure,JAKE,6,,757


In [17]:
enriched_movie1['Char'].value_counts()

JAKE           757
GRACE          434
QUARITCH       253
NEYTIRI        186
SELFRIDGE      137
NORM           123
(SUBTITLED)    119
TRUDY           93
MAX             69
MO'AT           63
TSU'TEY         55
WAINFLEET       33
Name: Char, dtype: int64

In [103]:
for i in files_list:
    print i
    title = i.split('/')[5].split('_')[0]
    genre = i.split('/')[4]
    #print genre
    title = title.strip()
    genre = genre.strip()
    movie = pd.read_fwf("%s" % i,header=None, names=["RawLine"],encoding='utf')
    print movie
    parsed = parse_movie(movie, genre, title)
    print parsed.head()
    final_df = pipeline(movie)
    print final_df.head()
    print "processing next movie"

./data/imsdb_scenes_dialogs_nov_2015/dialogs/Adventure/abyssthe_dialog.txt
                                               RawLine
0                                            THE ABYSS
1                               AN ORIGINAL SCREENPLAY
2                                                   BY
3                                        JAMES CAMERON
4                                       August 2, 1988
5                                  Director's Revision
6                                            THE ABYSS
7                                              CAPTAIN
8       Sixty knots?  No way, Barnes... the reds don't
9                             have anything that fast.
10                                              BARNES
11      Checked it twice, skipper.  It's a real unique
12      signature.  No cavitation, no reactor noise...
13                     doesn't even sound like screws.
14                                             CAPTAIN
15                                What the he

TypeError: cannot do label indexing on <class 'pandas.indexes.base.Index'> with these indexers [nan] of <type 'float'>

In [70]:
avatar = parse_movie(movie1, 'Adventure', 'avatar') 
print avatar.head()

                            RawLine   Title      Genre    Char  LineID  \
0                            AVATAR  avatar  Adventure  AVATAR       1   
1                        Written by  avatar  Adventure  AVATAR       1   
2                     James Cameron  avatar  Adventure  AVATAR       1   
3                      VOICE (V.O.)  avatar  Adventure   VOICE       2   
4  When I was lying there in the VA  avatar  Adventure   VOICE       2   

                             Speech  
0                                    
1                        Written by  
2                     James Cameron  
3                                    
4  When I was lying there in the VA  


In [65]:
movie1 = pd.read_fwf('./data/imsdb_scenes_dialogs_nov_2015/dialogs/Adventure/avatar_dialog.txt',
                    header=None, names=["RawLine"],encoding='utf')
print movie1

                                        RawLine
0                                        AVATAR
1                                    Written by
2                                 James Cameron
3                                  VOICE (V.O.)
4              When I was lying there in the VA
5       hospital, with a big hole blown through
6       the middle of my life, I started having
7                       these dreams of flying.
8                                  VOICE (V.O.)
9       Sooner or later though, you always have
10                                to wake up...
11                                      CUT TO:
12                                  JAKE (V.O.)
13     They can fix a spinal, if you've got the
14       money. But not on vet benefits, not in
15                                this economy.
16                             PERKY NEWSCASTER
17         The Bengal tiger, extinct for over a
18         century, is making a comeback. These
19         cloned tiger cubs at the Beij